# vLLM 연결 및 기능 테스트 노트북

이 노트북은 vLLM 서버의 OpenAI 호환 API 연결을 테스트하고, Tool Calling 및 멀티모달(이미지) 입력 기능을 검증합니다.

In [1]:
import os, sys
from dotenv import load_dotenv

load_dotenv()
sys.path.insert(0, os.path.abspath(".."))

print("환경 설정 및 경로 추가 완료")

환경 설정 및 경로 추가 완료


In [2]:
# 셀 1: vLLM 연결 테스트
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# LLM 설정 목록
llm_configs = {
    "runpod": {
        "base_url": "https://api.runpod.ai/v2/8iiebdj6zt0fbd/openai/v1",
        "model": "qwen/qwen3-vl-4b-thinking-fp8",
        "api_key": os.getenv("RUNPOD_API_KEY")
    },
    "local": {
        "base_url": "http://192.168.50.140:8000/v1",
        "model": "Qwen3-VL-2B-Thinking-FP8",
        "api_key": "EMPTY"
    }
}

# ──────────────────────────────────────────────────────────────────────
# 사용할 설정 선택 ("local" 또는 "runpod")
# ──────────────────────────────────────────────────────────────────────
target = "local"
cfg = llm_configs[target]

# 하위 셀 호환성을 위해 변수 할당
VLLM_BASE_URL = cfg["base_url"]
MODEL_NAME = cfg["model"]

print(f"현재 활성화된 설정: [{target}]")
print(f"Base URL: {VLLM_BASE_URL}")
print(f"Model Name: {MODEL_NAME}")

if VLLM_BASE_URL:
    llm = ChatOpenAI(
        base_url=VLLM_BASE_URL,
        model=MODEL_NAME,
        temperature=0,
        api_key=cfg["api_key"],
        max_tokens=5000
    )

    response = llm.invoke([
        HumanMessage(content="안녕! 너는 어떤 모델이니? 도구 호출이 가능하니?")
    ])
    print("\n[응답 결과]")
    print(response.content)
else:
    print(f"'{target}' 설정을 확인해 주세요.")

현재 활성화된 설정: [local]
Base URL: http://192.168.50.140:8000/v1
Model Name: Qwen3-VL-2B-Thinking-FP8

[응답 결과]
Okay, the user is asking, "안녕! 너는 어떤 모델이니? 도구 호출이 가능하니?" which translates to "Hello! What model are you? Can you call tools?"

First, I need to confirm my identity. I'm Qwen, the large language model developed by Tongyi Lab. So I should start by stating that clearly.

Next, they're asking if I can call tools. I should explain that I can handle various tasks, but I need to clarify that I don't have direct access to external tools like APIs or databases. However, I can use my own capabilities to answer questions, generate text, etc.

Wait, the user might be confused about what "tool calls" mean. Maybe they're referring to functions like API calls or specific tools. I should clarify that while I can't directly call external tools, I can use my language model capabilities to process information and provide answers.

I should structure the response: first confirm the model, then explain

# Tool Calling 능력 단계별 테스트

모델이 tool calling을 할 수는 있지만 불안정함이 확인됨.
파인튜닝 전에 **어디까지 복잡한 문제를 풀 수 있는지** 경계를 파악하여 학습 데이터 설계에 활용.

| Level | 테스트 내용 | 핵심 검증 포인트 |
|-------|-----------|----------------|
| 1 | 단순 이동 | 기본 tool call 1회 |
| 2 | 복합 이동 | 좌표 계산 + 다중 action |
| 3 | 다중 도구 동시 호출 | move + update_notepad 동시 |
| 4 | 조건 추론 + 도구 호출 | 장애물 우회 경로 추론 |
| 5 | 이미지 + 추론 + 도구 호출 | VLM 이미지 분석 → tool call |
| 6 | 멀티턴 시뮬레이션 | tool result → 다음 tool call 연쇄 |

In [3]:
# ── 레벨 테스트 Setup: 도구 정의 + 테스트 하네스 ──────────────
import json as _json
import time as _time
import base64 as _base64
from typing import Literal

from langchain_core.tools import tool
from langchain_core.messages import (
    HumanMessage, AIMessage, ToolMessage, SystemMessage,
)
from pydantic import BaseModel, Field

# ── 사파리 에이전트 동일 도구 정의 ────────────────────────────

class Action(BaseModel):
    direction: Literal["UP", "DOWN", "LEFT", "RIGHT"] = Field(
        description="이동 방향"
    )
    steps: int = Field(description="이동 칸수 (1~3)", ge=1, le=3)

class MoveArgs(BaseModel):
    actions: list[Action] = Field(description="최대 4개 행동을 순서대로 실행")

@tool(args_schema=MoveArgs)
def move(actions: list[Action]) -> str:
    """플레이어를 이동시킨다."""
    return _json.dumps({"moved": True})

class NotepadArgs(BaseModel):
    content: str = Field(description="메모장 전체를 덮어쓴다. 유지할 내용도 포함해서 작성해야 한다. 최대 2000자.")

@tool(args_schema=NotepadArgs)
def update_notepad(content: str) -> str:
    """메모장 전체를 덮어쓴다."""
    return _json.dumps({"status": "updated"})

class DeclareFoundArgs(BaseModel):
    target: str = Field(description="찾은 타겟 이름")

@tool(args_schema=DeclareFoundArgs)
def declare_found(target: str) -> str:
    """특정 타겟을 찾아서 도달했음을 선언한다."""
    return _json.dumps({"declared": True})

class DeclareDoneArgs(BaseModel):
    reason: str = Field(default="", description="미션 완료 사유")

@tool(args_schema=DeclareDoneArgs)
def declare_done(reason: str = "") -> str:
    """전체 미션이 완료되었음을 선언한다."""
    return _json.dumps({"done": True})

ALL_TOOLS = [move, update_notepad, declare_found, declare_done]

SYSTEM_MSG = (
    "너는 50x50 격자 맵 위의 사파리 에이전트야. "
    "반드시 제공된 도구를 사용해서 행동해. 텍스트로 답하지 말고 도구를 호출해."
)

# ── 테스트 하네스 ────────────────────────────────────────────

def run_level_test(
    level: int,
    title: str,
    messages: list,
    tools: list = None,
    validator=None,
) -> dict:
    """레벨 테스트 실행 및 결과 출력"""
    if tools is None:
        tools = ALL_TOOLS

    print(f"\n{'='*60}")
    print(f"  Level {level}: {title}")
    print(f"{'='*60}")

    llm_with_tools = llm.bind_tools(tools)

    start = _time.time()
    try:
        resp = llm_with_tools.invoke(messages)
        elapsed = _time.time() - start
    except Exception as e:
        elapsed = _time.time() - start
        print(f"\n  ❌ 호출 실패: {e}")
        return {
            "level": level, "title": title, "success": False,
            "reason": str(e), "tool_calls": [], "tokens": {}, "elapsed": elapsed,
        }

    # 토큰 사용량
    usage = resp.response_metadata.get("token_usage", {})
    tokens = {
        "prompt": usage.get("prompt_tokens", "?"),
        "completion": usage.get("completion_tokens", "?"),
        "total": usage.get("total_tokens", "?"),
    }

    # tool calls 파싱
    tool_calls = resp.tool_calls or []

    # 검증
    if validator:
        passed, reason = validator(resp, tool_calls)
    else:
        passed = len(tool_calls) > 0
        reason = "tool call 존재" if passed else "tool call 없음"

    status = "✅ PASS" if passed else "❌ FAIL"

    print(f"\n  상태: {status}")
    print(f"  사유: {reason}")
    print(f"  소요시간: {elapsed:.1f}s")
    print(f"  토큰: prompt={tokens['prompt']}, completion={tokens['completion']}, total={tokens['total']}")
    print(f"  Tool calls ({len(tool_calls)}개):")
    for i, tc in enumerate(tool_calls):
        print(f"    [{i}] {tc['name']}({_json.dumps(tc['args'], ensure_ascii=False)})")

    # thinking 부분 제거 후 응답 텍스트 출력
    if resp.content:
        content = resp.content
        if "</think>" in content:
            content = content.split("</think>")[-1].strip()
        if content:
            print(f"  응답 텍스트: {content[:200]}...")

    return {
        "level": level, "title": title, "success": passed,
        "reason": reason, "tool_calls": tool_calls,
        "tokens": tokens, "elapsed": elapsed,
    }

results = []  # 전체 결과 누적

print("✅ 테스트 하네스 및 도구 정의 완료")
print(f"사용 가능 도구: {[t.name for t in ALL_TOOLS]}")

✅ 테스트 하네스 및 도구 정의 완료
사용 가능 도구: ['move', 'update_notepad', 'declare_found', 'declare_done']


In [4]:
# ── Level 1: 단순 이동 ───────────────────────────────────────
# "오른쪽으로 2칸 이동해" → move([{RIGHT, 2}])
# 가장 기본적인 tool call 1회

def validate_l1(resp, tool_calls):
    if not tool_calls:
        return False, "tool call 없음"
    move_calls = [tc for tc in tool_calls if tc["name"] == "move"]
    if not move_calls:
        return False, "move 호출 없음"
    actions = move_calls[0]["args"].get("actions", [])
    if len(actions) == 1 and actions[0]["direction"] == "RIGHT" and actions[0]["steps"] == 2:
        return True, "정확한 이동 (RIGHT, 2)"
    return False, f"예상과 다른 이동: {actions}"

result = run_level_test(
    level=1,
    title="단순 이동",
    messages=[
        SystemMessage(content=SYSTEM_MSG),
        HumanMessage(content="오른쪽으로 2칸 이동해."),
    ],
    tools=[move],
    validator=validate_l1,
)
results.append(result)


  Level 1: 단순 이동

  상태: ✅ PASS
  사유: 정확한 이동 (RIGHT, 2)
  소요시간: 2.0s
  토큰: prompt=316, completion=321, total=637
  Tool calls (1개):
    [0] move({"actions": [{"direction": "RIGHT", "steps": 2}]})


In [5]:
# ── Level 2: 복합 이동 (좌표 계산 + 다중 action) ─────────────
# "(0,0)에서 (2,3)으로 가" → move([{RIGHT,2}, {UP,3}])
# 좌표 계산 + 여러 action을 하나의 tool call에 담기

def validate_l2(resp, tool_calls):
    if not tool_calls:
        return False, "tool call 없음"
    move_calls = [tc for tc in tool_calls if tc["name"] == "move"]
    if not move_calls:
        return False, "move 호출 없음"

    # 모든 move call의 actions를 합산하여 최종 좌표 계산
    x, y = 0, 0
    total_actions = 0
    for mc in move_calls:
        actions = mc["args"].get("actions", [])
        total_actions += len(actions)
        for a in actions:
            d, s = a["direction"], a["steps"]
            if d == "RIGHT": x += s
            elif d == "LEFT": x -= s
            elif d == "UP": y += s
            elif d == "DOWN": y -= s

    if x == 2 and y == 3:
        return True, f"정확한 경로 → (2,3), actions={total_actions}개"
    return False, f"도착 좌표 ({x},{y}) ≠ (2,3)"

result = run_level_test(
    level=2,
    title="복합 이동 (좌표 계산 + 다중 action)",
    messages=[
        SystemMessage(content=SYSTEM_MSG),
        HumanMessage(content=(
            "현재 위치는 (0,0)이고 (2,3)으로 이동해야 해. "
            "x축은 RIGHT/LEFT, y축은 UP/DOWN이야. "
            "move 도구를 한 번만 호출해서 이동해."
        )),
    ],
    tools=[move],
    validator=validate_l2,
)
results.append(result)


  Level 2: 복합 이동 (좌표 계산 + 다중 action)

  상태: ❌ FAIL
  사유: tool call 없음
  소요시간: 32.3s
  토큰: prompt=355, completion=5000, total=5355
  Tool calls (0개):
  응답 텍스트: Okay, let's see. The user wants to move from (0,0) to (2,3). The grid is 50x50, so coordinates are probably x and y. The problem says x-axis is RIGHT/LEFT and y-axis is UP/DOWN.

First, I need to figu...


In [6]:
# ── Level 3: 다중 도구 동시 호출 ─────────────────────────────
# "오른쪽 2칸 이동하고, 메모장에 기록해"
# → move(...) + update_notepad(...) 동시 호출

def validate_l3(resp, tool_calls):
    if not tool_calls:
        return False, "tool call 없음"
    names = [tc["name"] for tc in tool_calls]
    has_move = "move" in names
    has_notepad = "update_notepad" in names
    if has_move and has_notepad:
        return True, f"move + update_notepad 동시 호출 ({len(tool_calls)}개)"
    missing = []
    if not has_move:
        missing.append("move")
    if not has_notepad:
        missing.append("update_notepad")
    return False, f"누락된 도구: {missing}, 호출된 도구: {names}"

result = run_level_test(
    level=3,
    title="다중 도구 동시 호출",
    messages=[
        SystemMessage(content=SYSTEM_MSG),
        HumanMessage(content=(
            "오른쪽 2칸 이동하고, 메모장에 '목표 근처 도착'이라고 기록해."
        )),
    ],
    tools=[move, update_notepad],
    validator=validate_l3,
)
results.append(result)


  Level 3: 다중 도구 동시 호출

  상태: ✅ PASS
  사유: move + update_notepad 동시 호출 (2개)
  소요시간: 4.1s
  토큰: prompt=428, completion=662, total=1090
  Tool calls (2개):
    [0] move({"actions": [{"direction": "RIGHT", "steps": 2}]})
    [1] update_notepad({"content": "목표 근처 도착"})


In [7]:
# ── Level 4: 조건 추론 + 도구 호출 (장애물 우회) ─────────────
# "현재 (3,3)이고 왼쪽은 나무로 막혀있어. (5,1)로 가려면?"
# 장애물 우회 경로를 추론한 후 tool call

def validate_l4(resp, tool_calls):
    if not tool_calls:
        return False, "tool call 없음"
    move_calls = [tc for tc in tool_calls if tc["name"] == "move"]
    if not move_calls:
        return False, "move 호출 없음"

    # 전체 이동 경로에서 LEFT가 있으면 실패
    x, y = 3, 3
    has_left = False
    total_actions = 0
    for mc in move_calls:
        actions = mc["args"].get("actions", [])
        total_actions += len(actions)
        for a in actions:
            d, s = a["direction"], a["steps"]
            if d == "LEFT":
                has_left = True
            if d == "RIGHT": x += s
            elif d == "LEFT": x -= s
            elif d == "UP": y += s
            elif d == "DOWN": y -= s

    issues = []
    if has_left:
        issues.append("LEFT 사용 (막힌 방향)")
    if x != 5 or y != 1:
        issues.append(f"도착 ({x},{y}) ≠ (5,1)")

    if not issues:
        return True, f"장애물 우회 성공 → (5,1), actions={total_actions}개"
    return False, " / ".join(issues)

result = run_level_test(
    level=4,
    title="조건 추론 + 도구 호출 (장애물 우회)",
    messages=[
        SystemMessage(content=SYSTEM_MSG),
        HumanMessage(content=(
            "현재 (3,3)에 있어. 왼쪽(LEFT)은 나무로 막혀있어서 갈 수 없어. "
            "(5,1)로 이동해야 해. x축은 RIGHT/LEFT, y축은 UP/DOWN이야. "
            "막힌 방향을 피해서 move 도구로 이동해."
        )),
    ],
    tools=[move],
    validator=validate_l4,
)
results.append(result)


  Level 4: 조건 추론 + 도구 호출 (장애물 우회)

  상태: ❌ FAIL
  사유: tool call 없음
  소요시간: 32.5s
  토큰: prompt=379, completion=5000, total=5379
  Tool calls (0개):
  응답 텍스트: Okay, let's see. The user is at position (3,3) and needs to move to (5,1). The problem says that moving left is blocked by a tree, so they can't go left. They need to move right or up/down, but the di...


In [8]:
# ── Level 5: 이미지 + 추론 + 도구 호출 ───────────────────────
# 사파리 게임 스크린샷 + "빨간 원숭이한테 이동해"
# 이미지 분석 → 좌표 추정 → tool call

sample_img_path = "images/safari-sample.png"
with open(sample_img_path, "rb") as f:
    _b64 = _base64.b64encode(f.read()).decode()
_image_url = f"data:image/png;base64,{_b64}"
print(f"✅ 이미지 로드: {sample_img_path} (base64 {len(_b64)} bytes)")

def validate_l5(resp, tool_calls):
    if not tool_calls:
        return False, "tool call 없음"
    move_calls = [tc for tc in tool_calls if tc["name"] == "move"]
    if not move_calls:
        return False, "move 호출 없음"
    actions = move_calls[0]["args"].get("actions", [])
    return True, f"이미지 분석 → move 호출 (actions={len(actions)}개)"

result = run_level_test(
    level=5,
    title="이미지 + 추론 + 도구 호출",
    messages=[
        SystemMessage(content=SYSTEM_MSG),
        HumanMessage(content=[
            {"type": "text", "text": (
                "이 이미지는 사파리 격자 맵이야. "
                "빨간색 배경의 원숭이를 찾아서 그쪽으로 이동해. move 도구를 사용해."
            )},
            {"type": "image_url", "image_url": {"url": _image_url}},
        ]),
    ],
    tools=[move],
    validator=validate_l5,
)
results.append(result)

✅ 이미지 로드: images/safari-sample.png (base64 38864 bytes)

  Level 5: 이미지 + 추론 + 도구 호출

  상태: ❌ FAIL
  사유: tool call 없음
  소요시간: 32.7s
  토큰: prompt=575, completion=5000, total=5575
  Tool calls (0개):
  응답 텍스트: Okay, let's see. The user wants me to find the red monkey on the grid and move towards it. First, I need to locate the monkey. Looking at the image, there's a red square with a monkey inside it. The m...


In [9]:
# ── Level 6: 멀티턴 시뮬레이션 ───────────────────────────────
# 1턴 결과를 주고 다음 행동 결정
# tool result → 다음 tool call 연쇄

def run_level6():
    print(f"\n{'='*60}")
    print(f"  Level 6: 멀티턴 시뮬레이션")
    print(f"{'='*60}")

    tools = [move, update_notepad, declare_found]
    llm_with_tools = llm.bind_tools(tools)

    total_tokens = {"prompt": 0, "completion": 0, "total": 0}
    all_tool_calls = []
    turn_results = []  # True/False per turn
    start = _time.time()

    # ── Turn 1: 초기 이동 ──
    messages = [
        SystemMessage(content=SYSTEM_MSG),
        HumanMessage(content=(
            "현재 (0,0)에 있고 목표 동물은 (3,2) 근처에 있어. "
            "먼저 오른쪽으로 3칸 이동해."
        )),
    ]

    print("\n  ── Turn 1: 초기 이동 ──")
    resp1 = llm_with_tools.invoke(messages)
    tc1 = resp1.tool_calls or []
    all_tool_calls.extend(tc1)
    turn_results.append(bool(tc1))

    u1 = resp1.response_metadata.get("token_usage", {})
    for k in total_tokens:
        total_tokens[k] += u1.get(f"{'prompt' if k == 'prompt' else 'completion' if k == 'completion' else 'total'}_tokens", 0)

    for tc in tc1:
        print(f"    → {tc['name']}({_json.dumps(tc['args'], ensure_ascii=False)})")
    if not tc1:
        print("    ❌ tool call 없음")

    # Tool result → Turn 2
    messages.append(resp1)
    for tc in tc1:
        messages.append(ToolMessage(
            content=_json.dumps({
                "moved": True, "actualSteps": 3, "blocked": False,
                "pos": {"x": 3, "y": 0}, "onAnimal": None,
            }),
            tool_call_id=tc["id"],
        ))

    # ── Turn 2: 위쪽 이동 ──
    messages.append(HumanMessage(content="이동 완료. 현재 (3,0)이야. 위쪽으로 2칸 이동해."))

    print("\n  ── Turn 2: 추가 이동 ──")
    resp2 = llm_with_tools.invoke(messages)
    tc2 = resp2.tool_calls or []
    all_tool_calls.extend(tc2)
    turn_results.append(bool(tc2))

    u2 = resp2.response_metadata.get("token_usage", {})
    for k in total_tokens:
        total_tokens[k] += u2.get(f"{'prompt' if k == 'prompt' else 'completion' if k == 'completion' else 'total'}_tokens", 0)

    for tc in tc2:
        print(f"    → {tc['name']}({_json.dumps(tc['args'], ensure_ascii=False)})")
    if not tc2:
        print("    ❌ tool call 없음")

    # Tool result → Turn 3
    messages.append(resp2)
    for tc in tc2:
        messages.append(ToolMessage(
            content=_json.dumps({
                "moved": True, "actualSteps": 2, "blocked": False,
                "pos": {"x": 3, "y": 2}, "onAnimal": "원숭이",
            }),
            tool_call_id=tc["id"],
        ))

    # ── Turn 3: 동물 발견 → declare_found ──
    messages.append(HumanMessage(content=(
        "이동 결과: 현재 (3,2)이고 원숭이 위에 도착했어! "
        "타겟 발견을 선언해."
    )))

    print("\n  ── Turn 3: 발견 선언 ──")
    resp3 = llm_with_tools.invoke(messages)
    tc3 = resp3.tool_calls or []
    all_tool_calls.extend(tc3)
    turn_results.append(bool(tc3))

    u3 = resp3.response_metadata.get("token_usage", {})
    for k in total_tokens:
        total_tokens[k] += u3.get(f"{'prompt' if k == 'prompt' else 'completion' if k == 'completion' else 'total'}_tokens", 0)

    for tc in tc3:
        print(f"    → {tc['name']}({_json.dumps(tc['args'], ensure_ascii=False)})")
    if not tc3:
        print("    ❌ tool call 없음")

    elapsed = _time.time() - start

    # ── 검증 ──
    has_declare = any(tc["name"] == "declare_found" for tc in tc3)
    turns_ok = sum(turn_results)

    if turns_ok == 3 and has_declare:
        passed, reason = True, "3턴 모두 tool call 성공 + declare_found 호출"
    elif turns_ok == 3:
        passed, reason = True, "3턴 모두 tool call 성공 (declare_found 누락)"
    else:
        passed, reason = False, f"{turns_ok}/3 턴만 tool call 성공"

    status = "✅ PASS" if passed else "❌ FAIL"
    print(f"\n  상태: {status}")
    print(f"  사유: {reason}")
    print(f"  소요시간: {elapsed:.1f}s")
    print(f"  누적 토큰: prompt={total_tokens['prompt']}, completion={total_tokens['completion']}, total={total_tokens['total']}")
    print(f"  총 Tool calls: {len(all_tool_calls)}개")

    return {
        "level": 6, "title": "멀티턴 시뮬레이션", "success": passed,
        "reason": reason, "tool_calls": all_tool_calls,
        "tokens": total_tokens, "elapsed": elapsed,
    }

result = run_level6()
results.append(result)


  Level 6: 멀티턴 시뮬레이션

  ── Turn 1: 초기 이동 ──
    → move({"actions": [{"direction": "RIGHT", "steps": 3}]})

  ── Turn 2: 추가 이동 ──
    → move({"actions": [{"direction": "UP", "steps": 2}]})

  ── Turn 3: 발견 선언 ──
    → declare_found({"target": "monkey"})

  상태: ✅ PASS
  사유: 3턴 모두 tool call 성공 + declare_found 호출
  소요시간: 4.7s
  누적 토큰: prompt=1883, completion=729, total=2612
  총 Tool calls: 3개


In [10]:
# ── 결과 요약 ────────────────────────────────────────────────

print(f"\n{'='*60}")
print(f"  Qwen3-VL Tool Calling 능력 테스트 결과")
print(f"{'='*60}")

header = f"  {'Level':<8} {'테스트 내용':<32} {'결과':<6} {'토큰':>8} {'시간':>7}"
print(f"\n{header}")
print(f"  {'-'*8} {'-'*32} {'-'*6} {'-'*8} {'-'*7}")

for r in results:
    lvl = f"L{r['level']}"
    title = r.get("title", "")[:30]
    status = "PASS" if r["success"] else "FAIL"
    tok = str(r["tokens"].get("total", "?"))
    elapsed = f"{r['elapsed']:.1f}s"
    print(f"  {lvl:<8} {title:<32} {status:<6} {tok:>8} {elapsed:>7}")

passed = sum(1 for r in results if r["success"])
total = len(results)

print(f"\n  통과: {passed}/{total}")

# 실패한 레벨 분석
failed = [r for r in results if not r["success"]]
if failed:
    print(f"\n  ❌ 실패 레벨 분석:")
    for r in failed:
        print(f"    Level {r['level']}: {r['reason']}")

# 모델 한계점 요약
print(f"\n  {'─'*56}")
if passed == total:
    print("  🟢 모든 레벨 통과 — 기본 tool calling 능력 양호")
elif passed >= 4:
    print("  🟡 복잡한 추론이 필요한 레벨에서 실패 — 파인튜닝 시 해당 패턴 보강 필요")
elif passed >= 2:
    print("  🟠 기본 호출은 가능하나 다중 도구/추론에서 불안정 — 단계별 학습 데이터 필요")
else:
    print("  🔴 기본 tool calling 자체가 불안정 — 모델/서버 설정 재검토 필요")


  Qwen3-VL Tool Calling 능력 테스트 결과

  Level    테스트 내용                           결과           토큰      시간
  -------- -------------------------------- ------ -------- -------
  L1       단순 이동                            PASS        637    2.0s
  L2       복합 이동 (좌표 계산 + 다중 action)        FAIL       5355   32.3s
  L3       다중 도구 동시 호출                      PASS       1090    4.1s
  L4       조건 추론 + 도구 호출 (장애물 우회)           FAIL       5379   32.5s
  L5       이미지 + 추론 + 도구 호출                 FAIL       5575   32.7s
  L6       멀티턴 시뮬레이션                        PASS       2612    4.7s

  통과: 3/6

  ❌ 실패 레벨 분석:
    Level 2: tool call 없음
    Level 4: tool call 없음
    Level 5: tool call 없음

  ────────────────────────────────────────────────────────
  🟠 기본 호출은 가능하나 다중 도구/추론에서 불안정 — 단계별 학습 데이터 필요
